In [3]:
import numpy as np
import time
import datetime
import json
import pandas as pd

import mysql.connector
from mysql.connector import Error

from bokeh.plotting import *
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, LogColorMapper, ColorMapper, LogTicker, ColorBar, BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter
# from .chart_constants import (PLOT_FORMATS, ORANGE, BLUE, DARK_GRAY, AXIS_FORMATS, ORANGE_SHADOW, 
#                              FONT_PROPS_SM, FONT_PROPS_MD, FONT_PROPS_LG, GREEN)
from bokeh.io import show
import bokeh.palettes as bp

import urllib
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

In [4]:
json_url = urllib.request.urlopen('https://api.helioviewer.org/?action=getDataSources')
hv_keys = json.loads(json_url.read())

In [5]:
start_time=time.time()
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='hv',
                                         user='hv_varun',
                                         password='Helioviewer@2020')

    sql_select_Query = "SELECT filepath, date, sourceid FROM data WHERE sourceId=%s;"%hv_keys['SDO']['AIA']['1600']['sourceId']
#     sql_select_Query = "SELECT count(*) FROM data WHERE filepath LIKE '/AIA/1600/%';"
#     sql_select_Query = "SELECT * FROM data LIMIT 20;"
    cursor = connection.cursor()
    cursor.execute(sql_select_Query)
    records = cursor.fetchall()
    print("Total number of rows in data is: ", cursor.rowcount)

    print("\nPrinting each laptop record")
#     for row in records:
#         print("Index = ", row[0])
#         print("Location = ", row[1])
#         print("Filename  = ", row[2])
#         print("OBS_DATE  = ", row[3], "\n")

except Error as e:
    print("Error reading data from MySQL table", e)
finally:
    if (connection.is_connected()):
        connection.close()
        cursor.close()
        print("MySQL connection is closed", time.time()-start_time )

Total number of rows in data is:  7056352

Printing each laptop record
MySQL connection is closed 106.28920698165894


In [ ]:
hv = pd.DataFrame(records, columns=cursor.column_names)
hv = hv.sort_values('date').reset_index(drop=True)
hv

In [ ]:
ym_min = hv['date'].min().year + hv['date'].min().month/12
ym_max = hv['date'].max().year + hv['date'].max().month/12

In [ ]:
day, ym = np.meshgrid(range(1,33), np.arange(ym_min, ym_max+1.1/12, 1/12))
coverage = np.zeros((ym.shape))

In [ ]:
for i in range(coverage.shape[0]):
    for j in range(coverage.shape[1]):
        try:
            edge = datetime.datetime(int(round(ym[i,j]//1)), int(round(ym[i,j]%1*12)), day[i,j])
            coverage[i,j] = len(hv[(hv['date'] >= edge) & (hv['date'] < (edge + datetime.timedelta(days=1)))])
        except ValueError as V:
            coverage[i,j] = np.nan
#             print('ValueError', V, '::' ,ym[i,j],day[i,j], int(round(ym[i,j]//1)), int(round(ym[i,j]%1*12)), day[i,j])

In [ ]:
print("{:%b %d, %Y}".format(datetime.datetime(2003,1,1)))

In [159]:
year = pd.date_range(hv['date'].min(), hv['date'].max(), freq='M').to_period('M').to_timestamp()
year = year.year.astype(str) + ' ' + year.month_name()# (year.year+year.month/12)#.astype(str)
date = np.arange(1,32).astype(str)

hv_cov = pd.DataFrame(data=coverage[:122,:31], index=year, columns=date)
hv_cov.index.name='Year'
hv_cov.columns.name = 'Date'
hv_cov

Date,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,31
Year,,,,,,,,,,,,,,,,,,,,,
2010 June,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2122.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2010 July,36.0,0.0,0.0,0.0,0.0,0.0,2296.0,2224.0,3365.0,3266.0,...,0.0,0.0,0.0,2283.0,2348.0,813.0,1568.0,2341.0,2382.0,2336.0
2010 August,2303.0,2423.0,2210.0,2401.0,2365.0,2411.0,2681.0,2399.0,2412.0,2086.0,...,2449.0,2366.0,2050.0,2238.0,2205.0,2260.0,2402.0,361.0,2160.0,2197.0
2010 September,2099.0,2358.0,2399.0,2367.0,587.0,0.0,2279.0,2302.0,2230.0,2229.0,...,2223.0,1815.0,2296.0,2357.0,1559.0,2213.0,2327.0,2252.0,2247.0,NaN
2010 October,2411.0,2419.0,2336.0,2281.0,2388.0,2238.0,2236.0,2282.0,1832.0,1817.0,...,2398.0,2414.0,2403.0,2416.0,2031.0,2397.0,2305.0,2403.0,2355.0,2433.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020 March,1797.0,1798.0,1787.0,1798.0,1797.0,1798.0,1799.0,1800.0,1800.0,1798.0,...,1797.0,1799.0,1798.0,1798.0,1798.0,1796.0,1798.0,1798.0,1797.0,1788.0
2020 April,1799.0,1798.0,1797.0,1795.0,1796.0,1797.0,1797.0,1799.0,1798.0,1799.0,...,1798.0,1797.0,1798.0,1796.0,1797.0,1799.0,1785.0,1797.0,1796.0,NaN
2020 May,1794.0,1796.0,1799.0,1797.0,1796.0,1795.0,1797.0,1797.0,1798.0,1800.0,...,1797.0,1796.0,1796.0,1795.0,1786.0,1798.0,1794.0,1704.0,1796.0,1796.0


In [162]:
df = pd.DataFrame(hv_cov.stack(), columns=['coverage']).reset_index()
# df['Year2'] = df['Year'].dt.year.astype(str) + ' ' + df['Year'].dt.month_name().astype('str')# (year.year+year.month/12)#.astype(str)

df['Year']=df['Year'].astype(str)

years = hv_cov.index.values#.astype(str)
dates = hv_cov.columns.values.astype(str)

colors = bp.Viridis[256]# ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=np.nanmin(df.coverage), high=np.nanmax(df.coverage))

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

output_file('AIA1600_coverage.html')
p = figure(title="AIA 1600 Coverage ({0} - {1})".format(years[0], years[-1]),
#            x_axis_type='datetime',
           x_range=years, y_range=list(reversed(dates)),
           x_axis_location="above", plot_width=1400, plot_height=800,
           x_axis_label="Year Month", y_axis_label="Date",
           tooltips=[('Date', '@Year @Date'), ('#Data Files', '@coverage')],
           tools=TOOLS, toolbar_location='below')

# p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi / 3
p.xaxis.axis_label_text_font_size = "12pt"
p.yaxis.axis_label_text_font_size = "12pt"
p.xaxis.visible = True
p.xgrid.visible = True

p.xaxis.major_label_text_font_size = "7pt"
p.yaxis.major_label_text_font_size = "8pt"


p.rect(x="Year", y="Date", width=1, height=1,
       source=df,
       fill_color={'field': 'coverage', 'transform': mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="10px",
                     ticker=BasicTicker(desired_num_ticks=10),
                     formatter=PrintfTickFormatter(format="%d"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

show(p)      # show the plot

In [ ]:
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=coverage.min(), high=coverage.max())

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title="Helioviewer AIA 1600 Coverage",
           x_range=list(day[0,:]), y_range=list(ym[:,0]),
           x_axis_location="above", plot_width=900, plot_height=400,
           tools=TOOLS, toolbar_location='below',
           tooltips=[('date', '@day[0,:] @ym[:,0]'), ('count', '@coverage')])

In [ ]:
day[0,:].size

In [ ]:
N = day[0,:].size
x = day[0,:]#np.linspace(0, 10, N)
y = ym[:,0]#np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy)

p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], source=)
p.x_range.range_padding = p.y_range.range_padding = 0

# must give a vector of image data for image parameter
p.image(image=[d], x=0, y=0, dw=10, dh=10, palette="Spectral11", level="image")
p.grid.grid_line_width = 0.5

output_file("image.html", title="image.py example")

show(p)

In [ ]:
x=day[0,:]
y=ym

N = 500
x = np.linspace(0, 10, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy)

p = figure(tooltips=[("x", "$x"), ("y", "$ym"), ("value", "@image")])
p.x_range.range_padding = p.y_range.range_padding = 0

# must give a vector of image data for image parameter
p.image(image=[d], x=0, y=0, dw=12, dh=12, palette="Spectral11", level="image")
p.grid.grid_line_width = 0.5

output_file("image.html", title="image.py example")

show(p)

In [ ]:
N = 10
x = np.arange(0, N)
y = np.arange(0, N)
Z1 = np.random.rand(N,N)
Z2 = np.ones((N, N))
Z2[Z1<0.5] = np.nan

X, Y = np.meshgrid(x, y)
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
plt.pcolormesh(X, Y, Z1, cmap=plt.cm.Blues)
plt.colorbar()
plt.pcolormesh(X, Y, Z2, cmap=plt.cm.Reds, vmin=0,vmax=1,alpha=0.5)
ax.set_xlabel(r'$x$', fontsize=22)
ax.set_ylabel(r'$y$', fontsize=22)
plt.show()

In [ ]:
img.view(dtype=np.uint8)

In [ ]:
coverage.shape

In [ ]:
view=0

In [ ]:
N,M = coverage.shape
img = np.empty((N, M), dtype=np.uint32)
# view = img.view(dtype=np.uint8).reshape((N, M, 4))
# for i in range(N):
#     for j in range(M):
#         view[i, j, 0] = int(255 * i / N)
#         view[i, j, 1] = 158
#         view[i, j, 2] = int(255 * j / M)
#         view[i, j, 3] = 255

output_file("image_rgba.html")

p = figure(plot_width=400, plot_height=400, x_range=(0, 10), y_range=(0, 10))

p.image_rgba(image=[img], x=[0], y=[0], dw=[10], dh=[10])

show(p)

In [ ]:
# prepare some date
N = 300
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)

# output to static HTML file
output_file("linked_brushing.html")

# NEW: create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=350, height=350, title=None)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=350, height=350, title=None)
right.circle('x', 'y1', source=source)

# put the subplots in a gridplot
p = gridplot([[left, right]])

# show the results
show(p)

In [ ]:
# prepare some data
x = [0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
y0 = [i**2 for i in x]
y1 = [10**i for i in x]
y2 = [10**(i**2) for i in x]

# output to static HTML file
output_file("log_lines.html")

# create a new plot
p = figure(tools="pan,box_zoom,reset,save",
           y_axis_type="log", y_range=[0.001, 10**11], title="log axis example",
           x_axis_label='sections', y_axis_label='particles')

# add some renderers
p.line(x, x, legend_label="y=x")
p.circle(x, x, legend_label="y=x", fill_color="white", size=8)
p.line(x, y0, legend_label="y=x^2", line_width=3)
p.line(x, y1, legend_label="y=10^x", line_color="red")
p.circle(x, y1, legend_label="y=10^x", fill_color="red", line_color="red", size=6)
p.line(x, y2, legend_label="y=10^x^2", line_color="orange", line_dash="4 4")

# show the results
show(p)

In [ ]:
# prepare some data
N = 4000
x = np.random.random(size=N) * 100
y = np.random.random(size=N) * 100
radii = np.random.random(size=N) * 1.5
colors = ["#%02x%02x%02x" % (int(r), int(g), 150) for r, g in zip(50+2*x, 30+2*y)]

# output to static HTML file (with CDN resources)
output_file("color_scatter.html", title="color_scatter.py example", mode="cdn")

TOOLS = "crosshair,pan,wheel_zoom,box_zoom,reset,box_select,lasso_select"

# create a new plot with the tools above, and explicit ranges
p = figure(plot_height=500, plot_width=500, tools=TOOLS, x_range=(0, 100), y_range=(0, 100))

# add a circle renderer with vectorized colors and sizes
p.circle(x, y, radius=radii, fill_color=colors, fill_alpha=0.6, line_color=None)
# color_mapper = LogColorMapper(palette=colors, low=1, high=1e7)
# color_bar = ColorBar(color_mapper=color_mapper, ticker=LogTicker(),
#                      label_standoff=12, border_line_color=None, location=(0,0))

# p.add_layout(color_bar, 'right')

# show the results
show(p)

In [44]:
# prepare some data
N = 100
x = np.linspace(0, 4*np.pi, N)
y0 = np.sin(x)
y1 = np.cos(x)
y2 = np.sin(x) + np.cos(x)

# output to static HTML file
output_file("linked_panning.html")

# create a new plot
s1 = figure(width=250, plot_height=250, title=None)
s1.circle(x, y0, size=10, color="navy", alpha=0.5)

# NEW: create a new plot and share both ranges
s2 = figure(width=250, height=250, x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.triangle(x, y1, size=10, color="firebrick", alpha=0.5)

# NEW: create a new plot and share only one range
s3 = figure(width=250, height=250, x_range=s1.x_range, title=None)
s3.square(x, y2, size=10, color="olive", alpha=0.5)

# NEW: put the subplots in a gridplot
p = gridplot([[s1, s2, s3]], toolbar_location=None)

# show the results
show(p)